In [1]:
pip install opencv-python==3.4.18.65

  Using cached opencv_python-3.4.18.65-cp36-abi3-macosx_10_15_x86_64.whl.metadata (18 kB)
Using cached opencv_python-3.4.18.65-cp36-abi3-macosx_10_15_x86_64.whl (44.3 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.9.0
ERROR: Cannot uninstall opencv-python 4.9.0, RECORD file not found. Hint: The package was installed by conda.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import pyautogui
import matplotlib.pyplot as plt
import apriltag
import argparse

In [3]:
# init part
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

#***DEFAULT PARAMETERS***#

# Change thresholds (0 to 255) 
params.minThreshold = 0
params.maxThreshold = 255

#Area - area in terms of pixels
params.filterByArea = True
params.minArea = 300
params.maxArea = 10000
      
#Circularity - measures how close the blob to a circle
params.filterByCircularity = True
params.minCircularity = 0.5 #"cicularity" of a square = 0.785 , circle = 1.000
params.maxCircularity = 1.00 

#Color - 0 (select darker) to  255 (select lighter)
params.filterByColor = False
#params.minColor = 0 
#params.maxColor = 255 

#Convexity - (Area of the Blob / Area of it’s convex hull)
params.filterByConvexity = False
params.minConvexity = 0.87
params.maxConvexity = 1.00

#Inertia - how elongated a shape is, line = 0.000, circle = 1.000,
params.filterByInertia = False
params.minInertiaRatio = 0.95
params.maxInertiaRatio = 1.00

#Repeatability
params.minRepeatability=1

font = cv2.FONT_HERSHEY_SIMPLEX


# def blob_process(img, threshold, detector):
#     gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, img = cv2.threshold(gray_frame, threshold, 255, cv2.THRESH_BINARY)
#     img = cv2.erode(img, None, iterations=2)
#     img = cv2.dilate(img, None, iterations=4)
#     img = cv2.medianBlur(img, 5)
#     keypoints = detector.detect(img)
#     print(keypoints)
#     return keypoints


def nothing(x):
    pass

def main():
    screen_w, screen_h = pyautogui.size()
    eyeCap = cv2.VideoCapture(0)
    screenCap = cv2.VideoCapture(1)
    
    #pcw = 'Parameter Control Window'
    #cv2.namedWindow(pcw)
    slf = 'Screen Live Feed'
    cv2.namedWindow(slf)
    image = 'Pupil Tracking Live Feed'
    cv2.namedWindow(image)
    
    threshold = 20
    areaParam = 1000
    circularityParam = 100
    colorParam = 20
    convexityParam = 100
    inertiaParam = 100
    
    
    cursorSwitch = 'Cursor OFF/ON'
    cv2.createTrackbar(cursorSwitch, image, 0, 1, nothing)
    
    while True:
        #frame for eye
        _, eyeFrame = eyeCap.read()
        #frame for screen
        _, screenFrame = screenCap.read()

        #convert to grayscale
        gray_frame = cv2.cvtColor(screenFrame, cv2.COLOR_BGR2GRAY)

        #print("[INFO] detecting AprilTags...")
        options = apriltag.DetectorOptions(families="tag36h11")
        detector = apriltag.Detector(options)
        results = detector.detect(gray_frame)
        #print("[INFO] {} total AprilTags detected".format(len(results)))

        # loop over the AprilTag detection results
        for r in results:
            # extract the bounding box (x, y)-coordinates for the AprilTag
            # and convert each of the (x, y)-coordinate pairs to integers
            (ptA, ptB, ptC, ptD) = r.corners
            ptB = (int(ptB[0]), int(ptB[1]))
            ptC = (int(ptC[0]), int(ptC[1]))
            ptD = (int(ptD[0]), int(ptD[1]))
            ptA = (int(ptA[0]), int(ptA[1]))

            # draw the bounding box of the AprilTag detection
            cv2.line(screenFrame, ptA, ptB, (0, 255, 0), 2)
            cv2.line(screenFrame, ptB, ptC, (0, 255, 0), 2)
            cv2.line(screenFrame, ptC, ptD, (0, 255, 0), 2)
            cv2.line(screenFrame, ptD, ptA, (0, 255, 0), 2)

            # draw the center (x, y)-coordinates of the AprilTag
            (cX, cY) = (int(r.center[0]), int(r.center[1]))
            cv2.circle(screenFrame, (cX, cY), 5, (0, 0, 255), -1)

            # draw the tag family on the image
            tagFamily = r.tag_family.decode("utf-8")
            cv2.putText(screenFrame, tagFamily, (ptA[0], ptA[1] - 15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            #print("[INFO] tag family: {}".format(tagFamily))
            
            
        for r in results:
                print(r.decode("utf-8"))
                
        #calculate centroid
        if(len(results) == 4):
            validCenter = True
            #print(results[2].center[0])
            #print(results[0].center[0])
            height = abs(int(results[2].center[0]) + int(results[0].center[0]))
            #print(height)
            xoffset =  height / 2
            
            leftX1 = int(results[0].center[0])
            #print(leftX1)
            leftX2 = int(results[0].center[0])
            #print(leftX2)

            centerX = xoffset + leftX1
            #print("Center X")

            #print(centerX)
            a = abs(int(results[1].center[1]) - int(results[0].center[1]))
            b = abs(int(results[3].center[1]) - int(results[2].center[1]))
            
            
            #print(height)
            num = b + (2 * a)
            denom = 3 * (a + b)
            offset = int((num / denom) + height)
            #print(offset)

            if(a > b):
                centerY = int(results[1].center[0]) + offset 
            else:
                centerY = int(results[3].center[0]) + offset 
            
            #print(centerY)

                
        else:
            validCenter = False
        
        
        #Add text to frame to update displayed parameters
        eyeFrame = cv2.rectangle(eyeFrame,(0,0),(2000,60),(0,0,0),-5) #black background for white text
        eyeFrame = cv2.putText(eyeFrame, f'Min Area: {params.minArea}', (10, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Max Area: {params.maxArea}', (10, 55), font, 0.5, (255,255,255), 1)

        eyeFrame = cv2.putText(eyeFrame, f'Min Circ: {params.minCircularity}', (200, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Max Circ: {params.maxCircularity}', (200, 55), font, 0.5, (255,255,255), 1)

        eyeFrame = cv2.putText(eyeFrame, f'Color: {colorParam}', (400, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Convexity: {convexityParam}', (600, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Inertia: {inertiaParam}', (800, 15), font, 0.5, (255,255,255), 1)
        eyeFrame = cv2.putText(eyeFrame, f'Threshold: {inertiaParam}', (1000, 15), font, 0.5, (255,255,255), 1)

        #Update Parameters
        params.minArea = 1000
        params.maxArea = 5000
        params.minCircularity = 0.5 #circularityParam/100
        params.maxCircularity = 1 #circularityParam/100
        #params.minColor = 0 # no minColor
        #params.maxColor = 255 # no maxColor
        params.minConvexity = 0
        params.maxConvexity = 1
        params.minInertiaRatio = 0 #inertiaParam/100 
        params.maxInertiaRatio = 1 #inertiaParam/100
        params.minThreshold = 0
        params.maxThreshold = 255

        # Create a detector with the parameters

        ver = (cv2.__version__).split('.')
        if int(ver[0]) < 3 :
            detector = cv2.SimpleBlobDetector(params)
        else : 
            detector = cv2.SimpleBlobDetector_create(params)

        keypoints = detector.detect(eyeFrame)

        points = cv2.KeyPoint_convert(keypoints)
        pointsTuple = tuple(map(tuple,points))

        #print(keypointsfirst)
        #print(keypoints[1])
        
        #check if 
        cursorSwitchOn = cv2.getTrackbarPos(cursorSwitch, image)

        if pointsTuple:
            xcoord = int(pointsTuple[0][0])
            ycoord = int(pointsTuple[0][1])
            if cursorSwitchOn:
                pyautogui.moveTo(xcoord,ycoord)

            #eyeFrame = cv2.putText(frame, 'C', (xcoord, ycoord), font, 0.5, (0,0,255), 1)
            eyeFrame = cv2.circle(eyeFrame,(xcoord, ycoord), 5, (0,0,255), 5)
            
        #if validCenter:
        #    screenFrame = cv2.circle(screenFrame,(centerX, centerY), 5, (0,0,255), 5)
            
        eyeFrame = cv2.drawKeypoints(eyeFrame, keypoints, eyeFrame, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

        cv2.imshow(image, eyeFrame)
        cv2.imshow(slf, screenFrame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

/private/var/folders/8v/p7szzjzd7bx0jz_vlzbg0m700000gn/T/pip-install-wbqxel8q/apriltag_0c40b5eef5104d16ae78977fd022c006/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(
/private/var/folders/8v/p7szzjzd7bx0jz_vlzbg0m700000gn/T/pip-install-wbqxel8q/apriltag_0c40b5eef5104d16ae78977fd022c006/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(
/private/var/folders/8v/p7szzjzd7bx0jz_vlzbg0m700000gn/T/pip-install-wbqxel8q/apriltag_0c40b5eef5104d16ae78977fd022c006/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


AttributeError: 'Detection' object has no attribute 'decode'